<a href="https://colab.research.google.com/github/juanfernandev/Teamwork-Extended-Functions/blob/main/Scrapper_completo_Ferrer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Configurar opciones de Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar en modo sin cabeza
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Inicializar el navegador
driver = webdriver.Chrome(options=chrome_options)
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import time

# Configuración común
HEADERS = {"User-Agent": "Mozilla/5.0"}
FIELDS = [
    "url", "productID", "name", "description", "image", "brand",
    "price", "priceCurrency", "availability",
    "short_description", "product_reference",
    "long_description", "pharma_advice",
    "original_price", "current_price",
    "category", "category_parent"
]

# Configurar navegador headless
def setup_driver():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument('--window-size=1920x1080')
    return webdriver.Chrome(options=options)

# Función para extraer URLs de onclick
def extraer_url_onclick(onclick_val):
    match = re.search(r"setLocation\('([^']+)'\)", onclick_val or '')
    if match:
        return 'https://www.farmaferrer.com' + match.group(1)
    return ''

# Extraer todas las categorías
def extraer_categorias():
    driver = setup_driver()
    driver.get('https://www.farmaferrer.com/')
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.mega > li.nivel-0')))

    categorias_extraidas = []
    categorias = driver.find_elements(By.CSS_SELECTOR, 'ul.mega > li.nivel-0')

    for categoria in categorias:
        try:
            nombre_padre = categoria.find_element(By.TAG_NAME, 'a').get_attribute('textContent').strip()
        except:
            continue

        try:
            seccion = categoria.find_element(By.CSS_SELECTOR, 'section')
            bloques_l2 = seccion.find_elements(By.CSS_SELECTOR, 'ul.menu-l-2 > li')

            for bloque in bloques_l2:
                try:
                    subcat_l2_a = bloque.find_element(By.TAG_NAME, 'a')
                    subcat_l2_nombre = subcat_l2_a.get_attribute('textContent').strip()
                    href = subcat_l2_a.get_attribute('href')
                    onclick = subcat_l2_a.get_attribute('onclick')
                    subcat_l2_url = href if href else extraer_url_onclick(onclick)

                    categorias_extraidas.append({
                        'categoria_padre': nombre_padre,
                        'categoria': subcat_l2_nombre,
                        'url': subcat_l2_url
                    })

                    subsubs = bloque.find_elements(By.CSS_SELECTOR, 'ul.menu-l-3 > li > a')
                    for subsub in subsubs:
                        nombre_l3 = subsub.get_attribute('textContent').strip()
                        href_l3 = subsub.get_attribute('href')
                        onclick_l3 = subsub.get_attribute('onclick')
                        url_l3 = href_l3 if href_l3 else extraer_url_onclick(onclick_l3)

                        categorias_extraidas.append({
                            'categoria_padre': nombre_padre,
                            'categoria': nombre_l3,
                            'url': url_l3
                        })
                except:
                    continue
        except:
            continue

    driver.quit()
    return categorias_extraidas

# Obtener todas las páginas de una categoría
def get_all_pages_in_category(category_url):
    res = requests.get(category_url, headers=HEADERS)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "html.parser")

    pages = [category_url]
    paginacion = soup.select("div.paginacion-listado ul.pagination li a[href]")

    for link in paginacion:
        href = link.get("href")
        if href and href not in pages:
            # Normalizar URLs para evitar duplicados
            normalized = href.split('?')[0]  # Eliminar parámetros de consulta
            if normalized not in [p.split('?')[0] for p in pages]:
                pages.append(href)

    # Eliminar posibles duplicados manteniendo el orden
    seen = set()
    unique_pages = []
    for page in pages:
        key = page.split('?')[0]
        if key not in seen:
            seen.add(key)
            unique_pages.append(page)

    return unique_pages

# Obtener links de productos desde una página de categoría
def get_product_links_from_page(page_url):
    res = requests.get(page_url, headers=HEADERS)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "html.parser")
    productos = soup.select("div.element-producto a[href]")
    links = []
    for a in productos:
        href = a.get("href")
        if href and href.startswith("http"):
            links.append(href)
    return list(set(links))

# Extraer detalles de un producto
def extract_product_details(url, categoria_padre, categoria):
    res = requests.get(url, headers=HEADERS)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "html.parser")

    data = dict.fromkeys(FIELDS, "")
    data["url"] = url
    data["category_parent"] = categoria_padre
    data["category"] = categoria

    # JSON-LD
    for script in soup.find_all("script", type="application/ld+json"):
        try:
            json_data = json.loads(script.string.strip())
            if isinstance(json_data, list):
                for obj in json_data:
                    if obj.get("@type") == "Product":
                        json_data = obj
                        break
            elif json_data.get("@type") != "Product":
                continue

            data["productID"] = json_data.get("productID", "")
            data["name"] = json_data.get("name", "")
            data["description"] = json_data.get("description", "")

            image = json_data.get("image", "")
            if isinstance(image, list):
                data["image"] = image[0]
            elif isinstance(image, str):
                data["image"] = image.strip()

            brand = json_data.get("brand", "")
            if isinstance(brand, dict):
                data["brand"] = brand.get("name", "")
            elif isinstance(brand, str):
                data["brand"] = brand

            offers = json_data.get("offers", [])
            if isinstance(offers, dict):
                offers = [offers]
            if offers:
                offer = offers[0]
                data["price"] = offer.get("price", "")
                data["priceCurrency"] = offer.get("priceCurrency", "")
                data["availability"] = offer.get("availability", "")

            break
        except Exception as e:
            print(f"⚠️ Error leyendo JSON-LD en {url}: {e}")
            continue

    # Descripción corta + referencia
    info_divs = soup.select("div.informacion")
    if len(info_divs) >= 2:
        info_div = info_divs[1]
    else:
        info_div = info_divs[0] if info_divs else None

    if info_div:
        short = info_div.select_one("h4 span")
        if short:
            data["short_description"] = short.text.strip()
        ref = info_div.select_one("span.referencia-producto strong")
        if ref:
            data["product_reference"] = ref.text.strip()

    # Descripción larga
    long_desc = soup.select_one("div#info")
    if long_desc:
        data["long_description"] = long_desc.get_text(separator="\n", strip=True)

    # Consejo farmacéutico
    consejo = soup.select_one("div#consejo")
    if consejo:
        data["pharma_advice"] = consejo.get_text(separator="\n", strip=True)

    # Precios
    original_price = soup.select_one("h3 span.precio-original")
    if original_price:
        data["original_price"] = original_price.text.strip()

    current_price = soup.select_one("h3.precio span")
    if current_price:
        data["current_price"] = current_price.text.strip()

    return data

# Guardar en CSV
def save_to_csv(productos, filename="productos_farmaferrer.csv"):
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=FIELDS)
        writer.writeheader()
        for producto in productos:
            writer.writerow(producto)

# Función principal
def main(test_mode=False, test_category=None):
    # Extraer todas las categorías
    print("🔍 Extrayendo categorías...")
    categorias = extraer_categorias()

    if test_mode and test_category:
        # Modo test: usar solo la categoría de prueba
        categorias = [cat for cat in categorias if test_category in cat['url']]
        if not categorias:
            print(f"❌ No se encontró la categoría de prueba: {test_category}")
            return
        print(f"🔬 Modo TEST activado. Procesando solo: {test_category}")

    productos = []
    for i, cat in enumerate(categorias, 1):
        if test_mode:
            if test_category not in cat['url']:
                continue
            print(f"\n🔬 MODO TEST: Procesando 2 páginas de {cat['categoria_padre']} > {cat['categoria']}")

            todas_paginas = get_all_pages_in_category(cat['url'])
            paginas = todas_paginas[:2]
            print(f"  📑 Páginas a procesar: {len(paginas)}/{len(todas_paginas)}")

            for j, pagina_url in enumerate(paginas, 1):
                try:
                    links = get_product_links_from_page(pagina_url)
                    print(f"  📄 Página {j}: {len(links)} productos encontrados")

                    for k, link in enumerate(links, 1):
                        try:
                            print(f"    🛒 ({k}/{len(links)}) Extrayendo: {link.split('/')[-1]}")
                            detalles = extract_product_details(link, cat['categoria_padre'], cat['categoria'])
                            productos.append(detalles)
                            time.sleep(0.3)
                        except Exception as e:
                            print(f"    ❌ Error en producto: {str(e)}")
                            continue  # Continuar con el siguiente producto

                except Exception as e:
                    print(f"  ❌ Error en página {pagina_url}: {str(e)}")
                    continue  # Continuar con la siguiente página

            break  # Salir después de procesar la categoría de prueba

            # Verificar límite de productos DESPUÉS de procesar cada página
            if test_mode and len(productos) >= 15:
                break  # Salir del bucle de páginas si alcanzamos el límite

            # Este break está correctamente alineado con el for de páginas
            break  # Salir después de procesar la categoría de prueba



    # Guardar resultados
    if productos:
        filename = "productos_test.csv" if test_mode else "productos_farmaferrer.csv"
        save_to_csv(productos, filename)
        print(f"\n✅ Proceso completado. Datos guardados en {filename}")
        print(f"📊 Total de productos extraídos: {len(productos)}")
    else:
        print("\n❌ No se extrajeron productos")

if __name__ == "__main__":
    # Ejecutar en modo test con la categoría de desodorantes
    main(test_mode=True, test_category="https://www.farmaferrer.com/higiene/higiene-corporal/desodorantes")

🔍 Extrayendo categorías...
🔬 Modo TEST activado. Procesando solo: https://www.farmaferrer.com/higiene/higiene-corporal/desodorantes

🔬 MODO TEST: Procesando 2 páginas de Higiene > Desodorantes
  📑 Páginas a procesar: 2/2
  📄 Página 1: 10 productos encontrados
    🛒 (1/10) Extrayendo: be-desodorante-antitranspirante-72-h--50-ml
    🛒 (2/10) Extrayendo: desodorante-vichy-homme-48h
⚠️ Error leyendo JSON-LD en https://www.farmaferrer.com/desodorante-vichy-homme-48h: Invalid control character at: line 8 column 48 (char 424)
    🛒 (3/10) Extrayendo: martiderm-desodorante-48h-driosec-dermoprotect-roll-on-50-ml
    🛒 (4/10) Extrayendo: medicis-desod-roll-on
    🛒 (5/10) Extrayendo: lambda-control-desod-cr
    🛒 (6/10) Extrayendo: martiderm-drios-desod-roll-on
    🛒 (7/10) Extrayendo: isdindeo-lambda-control-48-h-50ml
⚠️ Error leyendo JSON-LD en https://www.farmaferrer.com/isdindeo-lambda-control-48-h-50ml: Invalid control character at: line 8 column 48 (char 317)
    🛒 (8/10) Extrayendo: lambd